In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from tf_unet import image_gen
from tf_unet import unet
from tf_unet import util
from tf_unet import image_util

generator = image_util.ImageDataProvider('../../train_test_data/test0/*[_img,_segmentation].tif', data_suffix=u'_img.tif', mask_suffix=u'_segmentation.tif')
net = unet.Unet(channels=generator.channels, n_class=generator.n_class, layers=4, features_root=20)

Number of files used: 200


2017-03-30 19:11:46,228 Layers 4, features 20, filter size 3x3, pool size: 2x2


In [2]:
import glob, re, os, warnings
import csv
from sklearn.metrics import jaccard_similarity_score, accuracy_score, precision_score
from skimage.io import imsave, imread
from skimage.transform import rescale
    
start_at = 0
end_at = 200
test_imgs = '../../train_test_data/test0/*[_img].tif'
train_dir = '../../ISIC-2017_Training_Part1_GroundTruth'
out_dir = './out_dir/'
prediction_model = "../../unet_models/train_4_20/train0_4_20_adam/unet_trained/model.cpkt"
out_size = 256

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

img_scores = []

num_processed = 0
files_list = sorted(glob.glob(test_imgs))
for file_name in files_list:
    if num_processed < start_at:
        num_processed += 1
        continue
        
    if num_processed > end_at:
        break
 
    img_name = re.match(r'.*/(ISIC_\d+)\_img.tif', file_name).group(1)
    print ("%s: %d/%d" % (img_name, num_processed, len(files_list)))
    
    img = imread(file_name)/255
    input_img = np.array([img])
    prediction = net.predict(prediction_model, input_img)
    prediction = prediction[0,...,1] > 0.5
    ground_truth = imread('%s/%s_segmentation.png' % (train_dir, img_name))/255
    
    offset = (prediction.shape[0] - out_size) // 2
    prediction[offset:-offset,offset:-offset]
    
    scale = np.max(ground_truth.shape)/out_size
    out_im = rescale(prediction, scale, order=0)
    out_im = out_im[0:ground_truth.shape[0], 0:ground_truth.shape[1]]
    
    ground_truth_flatten = np.expand_dims(ground_truth.flatten().astype(bool),axis=0)
    out_im_flatten = np.expand_dims(out_im.flatten().astype(bool), axis=0)
    jaccard_idx = jaccard_similarity_score(ground_truth_flatten, out_im_flatten)
    accuracy_idx = accuracy_score(ground_truth.flatten(), out_im.flatten())
    precision_idx = precision_score(ground_truth.flatten(), out_im.flatten())
    specivity_idx = precision_score(1 - ground_truth.flatten(), 1 - out_im.flatten())
    print(jaccard_idx, accuracy_idx, precision_idx, specivity_idx)
    img_scores.append((img_name, jaccard_idx, accuracy_idx, precision_idx, specivity_idx))
    
    imsave(('%s/%s_segmentation.png' % (out_dir, img_name)), out_im)
    
    num_processed += 1

print('writing jaccard score csv...\n')
with open('%s/jaccard.csv' % (out_dir), 'wb') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Image_Name', 'Jaccard', 'Accuracy', 'Precision', 'Specificity'])
    for value in img_scores:
       writer.writerow(value)
    
    avg_jaccard = sum(score[1] for score in img_scores)/float(len(img_scores))
    avg_accuracy = sum(score[2] for score in img_scores)/float(len(img_scores))
    avg_precision = sum(score[3] for score in img_scores)/float(len(img_scores))
    avg_specificity = sum(score[3] for score in img_scores)/float(len(img_scores))
    avg_scores = ['Average Score', avg_jaccard, avg_accuracy, avg_precision, avg_specificity]
    print("Average Scores: %r\n" % avg_scores)
    writer.writerow(avg_scores)
    
print('Done!\n')

ISIC_0000018: 0/200


2017-03-30 19:11:52,109 Model restored from file: ../../unet_models/train_hist_4_20/train0_hist/unet_trained/model.cpkt


0.378175184149 0.855673915911 1.0 0.841786783888
ISIC_0000047: 1/200


/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
2017-03-30 19:11:54,698 Model restored from file: ../../unet_models/train_hist_4_20/train0_hist/unet_trained/model.cpkt


0.530691510111 0.885218396058 1.0 0.868098290295
ISIC_0000069: 2/200


...

0.621531875565 0.869431642431 1.0 0.833792979308
ISIC_0015110: 198/200


2017-03-30 19:36:25,336 Model restored from file: ../../unet_models/train_hist_4_20/train0_hist/unet_trained/model.cpkt


0.768196681881 0.934643209669 0.905666967347 0.943751134268
ISIC_0015200: 199/200


2017-03-30 19:36:50,215 Model restored from file: ../../unet_models/train_hist_4_20/train0_hist/unet_trained/model.cpkt


0.113125331057 0.377335796485 1.0 0.323614592001
writing jaccard score csv...

Average Scores: ['Average Score', 0.33474690998415119, 0.87234119929855192, 0.84015698824894802, 0.84015698824894802]

Done!

